In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
pd.set_option('display.float_format', lambda x: '%.f' % x)

In [2]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
pd.set_option('display.float_format', lambda x: '%.f' % x)

## Read in Data ##
df = pd.read_csv('./datasources/raw/diabetic_data.csv')
mapping = pd.read_csv('./datasources/raw/IDs_mapping.csv')

fields = {}

fields['ID'] = ['encounter_id', 'admission_type_id', 'discharge_disposition_id',
                'admission_source_id', 'patient_nbr']

fields['DEMO'] = ['race', 'gender', 'age', 'weight']

fields['DIAG'] = ['diag_3', 'diag_2', 'diag_1']

fields['INT'] = ['num_medications', 'time_in_hospital', 'num_lab_procedures', 
                 'num_procedures', 'number_outpatient', 'number_emergency',
                 'number_inpatient', 'number_diagnoses']

fields['LAB'] = ['A1Cresult', 'max_glu_serum']

fields['MED'] = ['metformin', 'repaglinide', 'nateglinide', 'chlorpropamide',
                 'glimepiride', 'acetohexamide', 'glipizide', 'glyburide',
                 'tolbutamide', 'pioglitazone', 'rosiglitazone', 'acarbose',
                 'miglitol', 'troglitazone', 'tolazamide', 'examide', 'citoglipton',
                 'insulin', 'glyburide-metformin', 'glipizide-metformin', 
                 'glimepiride-pioglitazone', 'metformin-rosiglitazone', 
                 'metformin-pioglitazone', 'change', 'diabetesMed']

fields['TARGET'] = ['readmitted']

fields['OTHER'] = ['payer_code', 'medical_specialty']

In [3]:
## Filter out instances where patient was deceased at discharge ##
df = df[~df.discharge_disposition_id.isin([11,13,14,19,20,21])]

In [4]:
df.encounter_id.nunique() / df.patient_nbr.nunique()

1.4193884840691526

In [5]:
## Convert target variable (reamitted) to binary 1/0 for any reasmission type ##
df.readmitted = df.readmitted.apply(lambda x: 0 if x=='NO' else 1)

In [6]:
nulls = df.replace('?', np.nan).isnull().sum()
describe_df = pd.DataFrame(nulls, columns = ['# nulls'])
describe_df['# unique'] = [df[c].nunique() for c in nulls.index]
describe_df['unique values'] = [df[c].unique() for c in nulls.index]
describe_df['dtype'] = [df[c].dtypes for c in nulls.index]

In [7]:
describe_df.loc[fields['DEMO'], :]

,# nulls,# unique,unique values,dtype
race,2234,6,"[Caucasian, AfricanAmerican, ?, Other, Asian, ...",object
gender,0,3,"[Female, Male, Unknown/Invalid]",object
age,0,10,"[[0-10), [10-20), [20-30), [30-40), [40-50), [...",object
weight,96218,10,"[?, [75-100), [50-75), [0-25), [100-125), [25-...",object


In [8]:
describe_df.loc[fields['DIAG'], :]

,# nulls,# unique,unique values,dtype
diag_3,1419,787,"[?, 255, V27, 403, 250, V45, 38, 486, 996, 197...",object
diag_2,356,748,"[?, 250.01, 250, 250.43, 157, 411, 492, 427, 1...",object
diag_1,20,716,"[250.83, 276, 648, 8, 197, 414, 428, 398, 434,...",object


In [9]:
describe_df.loc[fields['INT'], :]

,# nulls,# unique,unique values,dtype
num_medications,0,75,"[1, 18, 13, 16, 8, 21, 12, 28, 17, 11, 15, 31,...",int64
time_in_hospital,0,14,"[1, 3, 2, 4, 5, 13, 12, 9, 7, 10, 6, 11, 8, 14]",int64
num_lab_procedures,0,118,"[41, 59, 11, 44, 51, 31, 70, 73, 68, 33, 47, 6...",int64
num_procedures,0,7,"[0, 5, 1, 6, 2, 3, 4]",int64
number_outpatient,0,39,"[0, 2, 1, 5, 7, 9, 3, 8, 4, 12, 11, 6, 20, 15,...",int64
number_emergency,0,33,"[0, 1, 2, 4, 3, 9, 5, 7, 6, 8, 22, 25, 10, 13,...",int64
number_inpatient,0,21,"[0, 1, 2, 3, 6, 5, 4, 7, 9, 8, 15, 10, 11, 14,...",int64
number_diagnoses,0,16,"[1, 9, 6, 7, 5, 8, 3, 4, 2, 16, 12, 13, 15, 10...",int64


In [10]:
describe_df.loc[fields['LAB'], :]

,# nulls,# unique,unique values,dtype
A1Cresult,0,4,"[None, >7, >8, Norm]",object
max_glu_serum,0,4,"[None, >300, Norm, >200]",object


In [11]:
describe_df.loc[fields['MED'], :].head()

,# nulls,# unique,unique values,dtype
metformin,0,4,"[No, Steady, Up, Down]",object
repaglinide,0,4,"[No, Up, Steady, Down]",object
nateglinide,0,4,"[No, Steady, Down, Up]",object
chlorpropamide,0,4,"[No, Steady, Down, Up]",object
glimepiride,0,4,"[No, Steady, Down, Up]",object


In [12]:
describe_df.loc[fields['OTHER'], :]

,# nulls,# unique,unique values,dtype
payer_code,39398,18,"[?, MC, MD, HM, UN, BC, SP, CP, SI, DM, CM, CH...",object
medical_specialty,48616,73,"[Pediatrics-Endocrinology, ?, InternalMedicine...",object


In [13]:
df.readmitted.value_counts()

0    52527
1    46816
Name: readmitted, dtype: int64

In [ ]:
def custom_plot(cols, fontsize=8, field_sort=False):

    fig = plt.figure(figsize=(16,4))
    for i, c in enumerate(cols[:4]):
        if field_sort:
            sort = c
        else:
            sort = 'encounter_id'
        cx0 = fig.add_subplot(140 + i+1)
        cx1 = cx0.twinx()

        grouped_df = df.replace(np.nan, 'null').groupby(c)\
                    .agg({'readmitted':'mean', 'encounter_id':'count'})\
                    .sort_values(sort)\
                    .reset_index()

        grouped_df['encounter_id'].plot(kind='bar', ax=cx0, fontsize=fontsize, title=c)
        grouped_df['readmitted'].plot(ax=cx1, color='orange',secondary_y=True, fontsize=fontsize)
        cx0.set_xticklabels(grouped_df[c], fontsize=12)
        plt.ylim(0,1)

    fig = plt.figure(figsize=(16,4))
    for i, c in enumerate(cols[4:8]):
        if field_sort:
            sort = c
        else:
            sort = 'encounter_id'
        cx0 = fig.add_subplot(140 + i+1)
        cx1 = cx0.twinx()

        grouped_df = df.replace(np.nan, 'null').groupby(c)\
                    .agg({'readmitted':'mean', 'encounter_id':'count'})\
                    .sort_values(sort)\
                    .reset_index()

        grouped_df['encounter_id'].plot(kind='bar', ax=cx0, fontsize=fontsize, title=c)
        grouped_df['readmitted'].plot(ax=cx1, color='orange',secondary_y=True, fontsize=fontsize)
        cx0.set_xticklabels(grouped_df[c], fontsize=fontsize)
        plt.ylim(0,1)

In [14]:
df = df.replace('?', np.nan)
weight_mapping = {'null':-1,
                    '[75-100)':75,   
                    '[50-75)':50,
                    '[100-125)':100,    
                    '[125-150)': 125, 
                    '[25-50)':25,         
                    '[0-25)':0,        
                    '[150-175)':150,       
                    '[175-200)':175,    
                    '>200':200}             
df = df.fillna('null')
df['weight_mod'] = df['weight'].apply(lambda x: weight_mapping[x])

age_mapping = {'[70-80)':70,
                '[60-70)': 60,
                '[50-60)': 50,
                '[80-90)': 80,
                '[40-50)': 40,
                '[30-40)':30,
                '[90-100)':90,
                '[20-30)':20,
                '[10-20)':10,
                '[0-10)':0}

df['age_mod'] = df['age'].apply(lambda x: age_mapping[x])

In [ ]:
custom_plot(['race', 'gender', 'age_mod', 'weight_mod'], fontsize=8, field_sort=True)

In [ ]:
custom_plot(fields['INT'], fontsize=8, field_sort=True)

In [15]:
dummy_fields = fields['DEMO']+fields['DIAG']+fields['OTHER']+fields['LAB']+fields['MED']
numeric_fields = fields['INT']

dummies = pd.get_dummies(df[dummy_fields])
X = pd.concat([df[numeric_fields] , dummies], axis=1)

y = df[fields['TARGET']]

In [ ]:
.shape

In [16]:
import scipy
from sklearn.feature_selection import VarianceThreshold
X_sparse = scipy.sparse.csc_matrix(X.values)
y_sparse = scipy.sparse.csc_matrix(y.values)

In [19]:
var_filter = VarianceThreshold(threshold=0.01)
X_var = var_filter.fit_transform(X_sparse)
cols = [i for i in X.columns[var_filter.get_support()]]

X_var.shape

(99343, 143)

In [24]:
X_var = pd.DataFrame(X_var.todense(), columns = cols)

In [26]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

chi_vals, p_vals = chi2(X_var, y)

In [27]:
chi_thres = 0
p_thres = 0.05
chi_filter = [True if c>=chi_thres else False for c in chi_vals]
p_filter = [True if p<=p_thres else False for p in p_vals]
stat_filter = np.logical_and(chi_filter, p_filter)

In [28]:
X = X_var.loc[:,stat_filter]

In [29]:
df = pd.concat([X.reset_index(drop=True), y.reset_index(drop=True)], axis=1)

In [30]:
df.readmitted.value_counts()

0    52527
1    46816
Name: readmitted, dtype: int64

In [31]:
df.shape

(99343, 106)

In [32]:
from sklearn.utils import resample

df_majority = df[df['readmitted']==0]
df_minority = df[df['readmitted']==1]

# Upsample minority class
df_majority_dnsampled = resample(df_majority, 
                                 replace=True,     # sample with replacement
                                 n_samples=len(df_minority),    # to match majority class
                                 random_state=42) # reproducible results

# Combine majority class with upsampled minority class
df_dnsampled = pd.concat([df_minority, df_majority_dnsampled])

# Display new class counts
df_dnsampled.readmitted.value_counts()

1    46816
0    46816
Name: readmitted, dtype: int64

In [33]:
X = df_dnsampled.drop('readmitted', axis=1)
y = df_dnsampled.readmitted

In [ ]:
import scipy
X_sparse = scipy.sparse.csc_matrix(X.values)
y_sparse = scipy.sparse.csc_matrix(y.values)
scipy.sparse.save_npz('./readmissions_X.npz', X_sparse)
scipy.sparse.save_npz('./readmissions_y.npz', y_sparse)